
# Árboles de Decisión — Banknote Authentication (UCI)

**Objetivos de la práctica**  
- Cargar el dataset  
- EDA  
- División *train/test* (70/30)  
- Entrenar un árbol de decisión  
- Comparar precisión en función de `max_depth`  
- Mostrar el árbol en **texto**  
- Visualizar **regiones de decisión** (con 2 *features*)  
- Construir **Matriz de Confusión** y métricas  
- Redactar una **interpretación final**


In [ ]:

# 0) Imports y configuración si no tienes alguno de ellos, usa en la Terminal: pip install (nombre de la librería)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score, precision_score,
    recall_score, f1_score
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

plt.rcParams['figure.figsize'] = (7,5)
plt.rcParams['axes.grid'] = True



## 1) Cargar dataset y quedarnos con 2 *features*
Usaremos **variance** y **skewness** para poder dibujar regiones de decisión fácilmente.


In [ ]:

# 1) Carga de datos
possible_paths = [
    'data_banknote_authentication.csv',
    '/kaggle/input/bank-note-authentication-uci-data/data_banknote_authentication.csv'
]

df = None
for p in possible_paths:
    try:
        df = pd.read_csv(p)
        break
    except Exception:
        pass

if df is None:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt'
    cols = ['variance','skewness','curtosis','entropy','target']
    df = pd.read_csv(url, header=None, names=cols)

df.columns = [c.lower().strip() for c in df.columns]
if 'class' in df.columns and 'target' not in df.columns:
    df.rename(columns={'class':'target'}, inplace=True)

X = df[['variance','skewness']].values
y = df['target'].values

df.head()


,variance,skewness,curtosis,entropy,target
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0



## 2) EDA
Comprobamos tamaños, valores nulos, balance de clases y estadísticas descriptivas.
#### Pista: Mira la API de la librería Pandas para hacer un mejor EDA


In [ ]:

print("Shape:", )

print("\nValores nulos por columna:")
print()

print("\nBalance de clases (target):")
print(df['target'].value_counts())

print("\nEstadísticas descriptivas (variance & skewness)")
display(df[['variance','skewness']].describe())

##### Dispersión
plt.figure()
plt.scatter(df['variance'], df['skewness'], alpha=0.3)
plt.xlabel('variance'); plt.ylabel('skewness'); plt.title('Dispersión: variance vs skewness')
plt.show()



## 3) División en *train/test* (70/30)


In [ ]:

#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=???, random_state=42, stratify=y
#)
#X_train.shape, X_test.shape



## 4) Árbol de decisión — modelo base


In [ ]:

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

y_pred_tr = tree.predict(X_train)
y_pred_te = tree.predict(X_test)

acc_tr = accuracy_score(y_train, y_pred_tr)
acc_te = accuracy_score(y_test, y_pred_te)

print(f"Accuracy (train): {acc_tr:.3f}")
print(f"Accuracy (test) : {acc_te:.3f}")


### Interpretación a priori de los Accuracy:

(deja aquí lo que interpretas de los accuracy obtenidos)



## 5) Precisión según `max_depth`


In [ ]:

depths = [1,2,3,4,5,6,7,8,9,10,None]
train_scores, test_scores = [], []

for d in depths:
    clf = DecisionTreeClassifier(random_state=42, max_depth=d)
    clf.fit(X_train, y_train)
    train_scores.append(clf.score(X_train, y_train))
    test_scores.append(clf.score(X_test, y_test))

x_labels = [str(d) if d is not None else 'None' for d in depths]

plt.figure()
plt.plot(range(len(depths)), train_scores, marker='o', label='Train')
plt.plot(range(len(depths)), test_scores, marker='o', label='Test')
plt.xticks(range(len(depths)), x_labels)
plt.xlabel('max_depth'); plt.ylabel('Accuracy')
plt.title('Profundidad del árbol vs Accuracy'); plt.legend(); plt.show()


### Interpretación los datos obtenidos:

(deja aquí lo que interpretas de los datos obtenidos)



## 6) Mostrar árbol en **texto** según variance y skewness (nuestras feature_names)
### Aquí elegiremos la mejor profundidad del árbol que observemos según el punto anterior y la pondremos en max_depth


In [ ]:

best_tree = DecisionTreeClassifier(random_state=42, max_depth=)
best_tree.fit(X_train, y_train)

# tree_text = export_text(best_tree, feature_names=['',''])
print(tree_text)



## 7) Regiones de decisión (2D)


In [ ]:

def plot_decision_regions_2d(model, X, y, title='Decision regions'):
    x_min, x_max = X[:, 0].min()-0.5, X[:, 0].max()+0.5
    y_min, y_max = X[:, 1].min()-0.5, X[:, 1].max()+0.5
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 300),
        np.linspace(y_min, y_max, 300)
    )
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid).reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.25)
    plt.scatter(X[:,0], X[:,1], c=y, edgecolor='k', alpha=0.8)
    plt.xlabel('variance'); plt.ylabel('skewness'); plt.title(title)
    plt.show()

plot_decision_regions_2d(best_tree, X_train, y_train, title='Regiones de decisión (train)')
plot_decision_regions_2d(best_tree, X_test, y_test, title='Regiones de decisión (test)')



## 8) Matriz de Confusión y métricas (calcula las principales métricas de la matriz de confusión: Accuracy, Precision, Recall, Specificity, FPR)
Filas = **Predicción** (Positivo/Negativo) · Columnas = **Realidad** (Positivo/Negativo)


In [ ]:

y_pred_best = best_tree.predict(X_test)

cm_official = confusion_matrix(y_test, y_pred_best, labels=[1,0])
TP, FN, FP, TN = cm_official.ravel()

cm_view = np.array([[TP, FP],
                    [FN, TN]])

brown_to_blue = LinearSegmentedColormap.from_list(
    "brown_to_blue",
    [
        (0.00, "#F4E1C6"),
        (0.35, "#CFA77B"),
        (0.60, "#9BA8B3"),
        (0.80, "#3C6EA6"),
        (1.00, "#0B3D91"),
    ],
    N=256
)

fig, ax = plt.subplots()
im = ax.imshow(cm_view, cmap=brown_to_blue)
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(['Positivo','Negativo'])
ax.set_yticklabels(['Positivo','Negativo'])
ax.set_xlabel('VALORES REALES'); ax.set_ylabel('VALORES PREDICCIÓN')
plt.colorbar(im, ax=ax)

for i in range(2):
    for j in range(2):
        ax.text(j, i, int(cm_view[i, j]), ha='center', va='center', color='white', fontsize=12)

plt.title('Matriz de Confusión')
plt.tight_layout(); plt.show()

#### Saca las métricas pedidas en el enunciado del punto 8. Pista: hay una función que te lo saca todo ;)



## 9) Preguntas finales:
- ¿En qué `max_depth` se logra **mejor rendimiento en test**? ¿Aparecerían señales de **sobreajuste** al aumentar la profundidad? ¿Por qué? 
- Observa la **matriz de confusión**: ¿qué tipo de error domina (FP o FN)? ¿Qué significa que predomine FP o FN?   
- Añade las otras observaciones que consideres importante
